Imports

In [1]:
import numpy as np
from sklearn.utils import resample
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
from matplotlib import style
from sklearn.metrics import classification_report, confusion_matrix, brier_score_loss, roc_curve, precision_recall_curve, silhouette_score, calinski_harabasz_score, davies_bouldin_score, recall_score, make_scorer, mean_squared_error
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.calibration import calibration_curve
from sklearn.model_selection import RandomizedSearchCV

Data Loading & Preprocessing

In [2]:
pulsar_dataset = '/home/d.dasarathan/ds5500/projects/datasets/HTRU2/HTRU_2.csv'
pulsar_data = np.loadtxt(pulsar_dataset, delimiter = ',', skiprows=0) 

In [3]:
def upsampling(X_train, y_train):
    X_minority = np.array([X_train[i] for i in range(len(X_train)) if y_train[i] == 1])
    kappa = (1.0 * len(X_minority)) / len(X_train)
    print('total training size: ', len(X_train))
    print('positive class training size: ', len(X_minority))
    repeat = int(1.0 / kappa) - 1
    upsampling = resample(X_minority, n_samples=repeat * len(X_minority))
    y_minority = np.ones(repeat * len(X_minority))
    X_train = np.concatenate((X_train, upsampling))
    print('updated positive class training size: ', len(upsampling))
    y_train = np.concatenate((y_train, y_minority))

In [4]:
def upsampling_train(X_train, y_train):
    X_minority = np.array([X_train[i] for i in range(len(X_train)) if y_train[i] == 1])
    kappa = (1.0 * len(X_minority)) / len(X_train)
    print('total training size: ', len(X_train))
    print('positive class training size: ', len(X_minority))
    repeat = int(1.0 / kappa) - 1
    upsampling = resample(X_minority, n_samples=repeat * len(X_minority))
    y_minority = np.ones(repeat * len(X_minority))
    X_train = np.concatenate((X_train, upsampling))
    print('updated positive class training size: ', len(upsampling))
    y_train = np.concatenate((y_train, y_minority))
    return X_train, y_train

In [5]:
def data_prep(X, y, normalize=False, upsample=False, train_val_test=False, train_test=False, k_fold=False, k=None):
  if normalize:
    scaler = StandardScaler().fit(X, y)
    X = scaler.transform(X)
  if train_val_test:
    X, X_test, y, y_test = train_test_split(X, y, test_size=0.25)
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25)
    if upsample:
      upsampling(X_train, y_train)
    return X_train, X_val, X_test, y_train, y_val, y_test
  if train_test:
    X, X_test, y, y_test = train_test_split(X, y, test_size=0.2)
    if k_fold:
      data_sets = []
      kf = KFold(n_splits=k, shuffle=True)
      for train_index, val_index in kf.split(X, y):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]
        if upsample: 
          upsampling(X_train, y_train)
        data_sets.append((X_train, X_val, y_train, y_val))
      if upsample:
        upsampling(X, y)
      return data_sets, X, X_test, y, y_test
    else:
      if upsample: 
        upsampling(X, y)
      return X, X_test, y, y_test
  if k_fold:
    data_sets = []
    kf = KFold(n_splits=k, shuffle=True)
    for train_index, test_index in kf.split(X, y):
      X_train, X_test = X[train_index], X[test_index]
      y_train, y_test = y[train_index], y[test_index]
      if upsample: 
        upsampling(X_train, y_train)
      data_sets.append((X_train, X_test, y_train, y_test))
    return data_sets, X, y
  return X, y

Display & Plot Evaluation Metrics (Classification)

In [6]:
def eval_clf(clf_pred, clf_pred_prob, y_val, clf_score, alternative_treshold=None, plot=False):
  print("score: ", clf_score)
  print(classification_report(y_val, clf_pred))
  cm = confusion_matrix(y_val, clf_pred)
  print('Confusion matrix\n', cm)
  print('True Positives(TP) = ', cm[1,1])
  print('True Negatives(TN) = ', cm[0,0])
  print('False Positives(FP) = ', cm[0,1])
  print('False Negatives(FN) = ', cm[1,0])

  TP = cm[1,1]
  TN = cm[0,0]
  FP = cm[0,1]
  FN = cm[1,0]
  
  ## classification accuracy
  classification_accuracy = (TP + TN) / float(TP + TN + FP + FN)
  print('Classification accuracy : {0:0.4f}'.format(classification_accuracy))

  ## classification error
  classification_error = (FP + FN) / float(TP + TN + FP + FN)
  print('Classification error : {0:0.4f}'.format(classification_error))

  ## precision score
  precision = TP / float(TP + FP)
  print('Precision : {0:0.4f}'.format(precision))

  ## recall 
  recall = TP / float(TP + FN)
  print('Recall or Sensitivity : {0:0.4f}'.format(recall))

  ## specificity
  specificity = TN / (TN + FP)
  print('Specificity : {0:0.4f}'.format(specificity))

  ## F-1 score
  f1 = 2 * (precision * recall) / (precision + recall)
  print('F-1 score : {0:0.4f}'.format(f1))
  
  ROC_AUC = roc_auc_score(y_val, clf_pred)
  print('ROC AUC : {:.4f}'.format(ROC_AUC))
    
  #print("brier score loss: ", brier_score_loss(y_val, clf_pred_prob[:,1]))

  if plot:
    clf_fpr, clf_tpr, clf_thresholds = roc_curve(y_val, clf_pred_prob[:,1])
    plt.plot(clf_fpr, clf_tpr, marker='.')
    plt.title('ROC Curve')
    plt.xlabel('1 - Specifity (FPR)')
    plt.ylabel('Sensitivity (TPR)')
    plt.show()
    plt.plot(clf_thresholds, clf_tpr, marker='.', label='Sensitivity (TPR)')
    plt.plot(clf_thresholds, clf_fpr, marker='.', label='1 - Specifity (FPR)')
    plt.plot(clf_thresholds, clf_tpr - clf_fpr, marker='.', label='Difference (TPR - FPR)')
    plt.xlim([0.0, 1.0])
    plt.title('Sensitivity / 1 - Specifity / Difference vs. Threshold')
    plt.xlabel('Threshold')
    plt.ylabel('Sensitivity / 1 - Specifity / Difference')
    plt.legend()
    plt.show()

    clf_precision, clf_recall, clf_thresholds = precision_recall_curve(y_val, clf_pred_prob[:,1])
    plt.plot(clf_recall, clf_precision, marker='.')
    plt.title('PRC Curve')
    plt.xlabel('Recall (TPR)')
    plt.ylabel('Precision (PPV)')
    plt.show()

    clf_frac_pos, clf_prob_pos = calibration_curve(y_val, clf_pred_prob[:,1], n_bins=10, strategy='uniform')
    plt.plot(clf_prob_pos, clf_frac_pos, marker='.')
    plt.title('Calibration')
    plt.xlabel('Probability')
    plt.ylabel('Fraction of Positives')
    plt.show()

  if alternative_treshold:
    print("Threshold = ", alternative_treshold, ":")
    clf_new_pred = [1 * (clf_pred_sample_prob[1] > alternative_treshold) for clf_pred_sample_prob in clf_pred_prob]
    print(classification_report(y_val, clf_new_pred))
    print("confusion matrix:")
    print(confusion_matrix(y_val, clf_new_pred))

Guassian Discriminant Analysis (Baseline)
(using scikit)

In [7]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
def gda(X_train, X_val, y_train, y_val):
  clf = QuadraticDiscriminantAnalysis()
  clf.fit(X_train, y_train)
  gda_pred = clf.predict(X_val)
  gda_pred_prob = clf.predict_proba(X_val)
  gda_score = clf.score(X_val, y_val)
  gda_train_error = mean_squared_error(y_train, clf.predict(X_train))
  gda_val_error = mean_squared_error(y_val, gda_pred)
  return gda_pred, gda_pred_prob, gda_score, gda_train_error, gda_val_error

Main

In [8]:
X, y = pulsar_data[:,:-1], pulsar_data[:,-1]
print("X shape : ",X.shape )
print("y shape : ",y.shape )
X_train, X_test, y_train, y_test = data_prep(X, y, train_test=True, upsample=False)
print("X_train shape : ",X_train.shape )
print("X_test shape : ",X_test.shape )

print("GDA - Test Set performance:")
gda_pred, gda_pred_prob, gda_score, gda_train_error, gda_val_error = gda(X_train, X_test, y_train, y_test)
print("recall: ", recall_score(y_test, gda_pred))
eval_clf(gda_pred, gda_pred_prob, y_test, gda_score, plot=False)

print("============================================================================================")

X, y = pulsar_data[:,:-1], pulsar_data[:,-1]
print("X shape : ",X.shape )
print("y shape : ",y.shape )

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print("X_train shape : ",X_train.shape )
print("X_test shape : ",X_test.shape )

X_train, y_train = upsampling_train(X_train, y_train)
print("X_train shape : ",X_train.shape)
print("y_train shape : ",y_train.shape)
print("X_test shape : ",X_test.shape)
print("y_test shape : ",y_test.shape)

np.set_printoptions(suppress=True)
unique, counts = np.unique(y_train, return_counts=True)
print("Train data class distribution")
print(np.asarray((unique, counts)).T)

unique, counts = np.unique(y_test, return_counts=True)
print("Test data class distribution")
print(np.asarray((unique, counts)).T)

print("GDA - Upsampled - Test Set Performance:")
gda_pred, gda_pred_prob, gda_score, gda_train_error, gda_test_error = gda(X_train, X_test, y_train, y_test)
print("recall: ", recall_score(y_test, gda_pred))
eval_clf(gda_pred, gda_pred_prob, y_test, gda_score, plot=False)

X shape :  (17898, 8)
y shape :  (17898,)
X_train shape :  (14318, 8)
X_test shape :  (3580, 8)
GDA - Test Set performance:
recall:  0.8640226628895185
score:  0.9675977653631285
              precision    recall  f1-score   support

         0.0       0.99      0.98      0.98      3227
         1.0       0.82      0.86      0.84       353

    accuracy                           0.97      3580
   macro avg       0.90      0.92      0.91      3580
weighted avg       0.97      0.97      0.97      3580

Confusion matrix
 [[3159   68]
 [  48  305]]
True Positives(TP) =  305
True Negatives(TN) =  3159
False Positives(FP) =  68
False Negatives(FN) =  48
Classification accuracy : 0.9676
Classification error : 0.0324
Precision : 0.8177
Recall or Sensitivity : 0.8640
Specificity : 0.9789
F-1 score : 0.8402
ROC AUC : 0.9215
X shape :  (17898, 8)
y shape :  (17898,)
X_train shape :  (14318, 8)
X_test shape :  (3580, 8)
total training size:  14318
positive class training size:  1298
updated positi